In [ ]:
import json
import os
import re
import glob
from collections import Counter
import random

In [ ]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy import sparse
import seaborn as sns
from gensim.models import Word2Vec
from math import log10
from sklearn.decomposition import PCA

from nltk import SnowballStemmer
from pymorphy3 import MorphAnalyzer
from nltk.corpus import stopwords


In [ ]:
DATASET_INPUT_DIR = os.path.realpath("./assets/data/annotaed-tsv/")
DATASET_OUTPUT_DIR = os.path.realpath("./assets/wordcount/train")
DATASET_EMBEDDINGS_TEST_DIR = os.path.realpath("./assets/embeddings/test/")
DATASET_EMBEDDINGS_TRAIN_DIR = os.path.realpath("./assets/embeddings/train/")
MODEL_ARTIFACTS_DIR = os.path.realpath("./assets/artifacts/")

TOKEN_FREQS_JSON_PATH = os.path.join(DATASET_OUTPUT_DIR, "token_frequencies.json")
TERM_DOC_PATH = os.path.join(DATASET_OUTPUT_DIR, "term_document.npz")
TOKEN_ORDER_PATH = os.path.join(DATASET_OUTPUT_DIR, "token_order.json")

DOCS_TRAIN_TEST_NAMES_FILE = os.path.join(DATASET_INPUT_DIR, "train_test_lists.json")
LOAD_TRAIN_TEST_FROM_JSON = True

DATASET_LANG = "russian"

In [ ]:
docs_file_paths = glob.glob(os.path.join(DATASET_INPUT_DIR, "*/*.tsv"))
docs_file_paths

In [ ]:
if LOAD_TRAIN_TEST_FROM_JSON:
    with open(DOCS_TRAIN_TEST_NAMES_FILE) as f:
        dataset_train_test_config = json.load(f)

    docs_file_paths_test = dataset_train_test_config["test"]
    docs_file_paths_train = dataset_train_test_config["train"]
else:
    docs_file_paths_test = set(random.sample(docs_file_paths, k=int(len(docs_file_paths) * 0.3)))
    docs_file_paths_train = list(set(docs_file_paths) - docs_file_paths_test)
    docs_file_paths_test = list(docs_file_paths_test)
    
    with open(DOCS_TRAIN_TEST_NAMES_FILE, "w") as f:
        json.dump({"train": docs_file_paths_train, "test": docs_file_paths_test}, f)

In [ ]:
phone_number_regex = r"(\+\d{1,3})?\s?\(?\d{1,4}\)?[\s.-]?\d{3}[\s.-]?\d{4}"
email_regex = r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,7}"
word_regex = r"([А-Яа-яЁёA-Za-z0-9-]+)"
token_pattern = re.compile("|".join([
    email_regex,
    phone_number_regex,
    word_regex,
]))

def get_stems_processed(path: str) -> list[list[str]]:
    sentences = []
    with open(path) as f:
        lines = "".join(f.readlines())
        sentences_raw = lines.split("\n\t\t\n")
        for sentence in sentences_raw:
            stems = []
            words = sentence.split("\n")
            if len(words) == 0 or words[0] == "":
                continue
            stems_raw = list(map(lambda x: x.split("\t")[1], words))
            lemmas = list(map(lambda x: x.split("\t")[2], words))
            for i in range(len(stems_raw)):
                if lemmas[i] not in stopwords.words(DATASET_LANG) and token_pattern.match(lemmas[i]) is not None:
                    stems.append(stems_raw[i])
            sentences.append(stems)
    return sentences

In [ ]:
def count_words_in_docs(sentences) -> list[dict[str, int]]:
    sentences_count = []
    for sentence in sentences:
        token_by_sent_count = Counter()
        for word in sentence:
            token_by_sent_count[word] += 1
        sentences_count.append(token_by_sent_count)
    return sentences_count

def count_words_in_docs(sentences: list[list[str]], count_by_sentences=False) -> dict[str, int] | list[dict[str, int]]:
    if count_by_sentences:
        return count_words_in_docs(sentences)
    
    token_count = Counter()
    for sentence in sentences:
        for word in sentence:
            token_count[word] += 1
    return token_count

In [ ]:
def count_all_stems(docs_file_paths: list[str]) -> tuple[dict[str, int], list[dict[str, int]], list[list[list[str]]]]:
    docs = []
    token_freqs = Counter()
    token_freq_by_doc = []
    
    for filename in tqdm(docs_file_paths):
        sents = get_stems_processed(filename)
        docs.append(sents)
        counts = count_words_in_docs(sents)

        token_freq_by_doc.append(counts)
        token_freqs.update(counts)

    return token_freqs, token_freq_by_doc, docs

In [ ]:
def filter_freqs(freqs: dict[str, int], threshhold: int) -> dict[str, int]:
    return {token: freq for token, freq in freqs.items() if freq >= threshhold}

In [ ]:
def save_freqs(freqs: dict[str, int], path: str) -> None:
    dir = "".join(path.split("/")[:-1])
    os.makedirs(dir, exist_ok=True)
    with open(path, "w") as f:
        json.dump(freqs, f)

In [ ]:
token_freqs, token_freqs_by_doc, all_docs = count_all_stems(docs_file_paths_train)
token_freqs = filter_freqs(token_freqs, 5)

token_freqs_by_doc = [{token: freq for token, freq in doc_freqs.items() if token_freqs.get(token, False)} for doc_freqs in token_freqs_by_doc]

In [ ]:
save_freqs(token_freqs, TOKEN_FREQS_JSON_PATH)

In [ ]:
def make_term_doc_matrix(token_freqs: dict[str, int], token_freqs_by_doc: list[dict[str, int]]) -> tuple[np.matrix, dict[str, int]]:
    raw_mat = np.zeros((len(token_freqs_by_doc), len(token_freqs)), dtype=np.uint16)
    term_dict = {token: id for id, token in enumerate(token_freqs.keys())}

    for doc_i, doc_token_freqs in enumerate(token_freqs_by_doc):
        for token, freq in doc_token_freqs.items():
            raw_mat[doc_i][term_dict[token]] = freq
    
    return raw_mat, term_dict

In [ ]:
def save_sparse_mat(matrix: np.ndarray, path: str) -> None:
    sp_mat = sparse.bsr_array(matrix, dtype=matrix.dtype)
    sparse.save_npz(path, sp_mat)

def load_sparse_mat(path: str, ) -> np.ndarray:
    sp_mat: sparse.sparray = sparse.load_npz(path)
    return sp_mat.toarray()

def save_token_ordering_dict(order_dict: dict[str, int], path: str) -> None:
    with open(path, "w") as f:
        json.dump(order_dict, f)

def load_ordering_dict(path: str) -> dict[str, int]:
    with open(path) as f:
        order_dict = json.load(f)
    return order_dict

In [ ]:
term_doc_martix, term_doc_token_id_dict = make_term_doc_matrix(token_freqs, token_freqs_by_doc)

In [ ]:
save_sparse_mat(term_doc_martix, TOKEN_ORDER_PATH)
save_token_ordering_dict(term_doc_token_id_dict, TOKEN_ORDER_PATH)

In [ ]:
matrix = pd.DataFrame(data=term_doc_martix, columns=sorted(term_doc_token_id_dict, key=term_doc_token_id_dict.get))
matrix.head()

In [ ]:
def tf_idf(words: dict[str, int], matrix: pd.DataFrame) -> list[float]:
    total_words = sum(words.values())
    total_documents = len(matrix.index)
    result = []
    for w in matrix.columns:
        if w not in words:
            result.append(0.0)
            continue
        t_f = words[w] / total_words
        d_f = sum(matrix[w] > 0)
        tfidf = t_f * (log10(total_documents + 1) - log10(d_f + 1))
        result.append(tfidf)
    return result

In [ ]:
def split_to_sentences(text: str) -> list[str]:
    sentences = re.split(
        r"(((?<!\w\.\w.)(?<!\s\w\.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s(?=[A-Z]))|((?<![\,\-\:])\n(?=[A-Z]|\" )))", text)[
                ::4]
    return sentences


def split_to_words(sentence: str) -> list[str]:
    words = re.findall(r"\w+@\w+\.\w+|\+\d{1,3}-\d{3}-\d{3}-\d{2}-\d{2}|\w+", sentence)
    return words

In [ ]:
def preprocess_text(text: str, by_sentences=False) -> list[str] | list[list[str]]:
    stemmer = SnowballStemmer(DATASET_LANG)
    lemmatizer = MorphAnalyzer()
    sentences = split_to_sentences(text)
    result = []
    for s in sentences:
        sentence = []
        for w in split_to_words(s):
            w_processed = re.sub(r"[.!?,]$", "", w).lower()
            lemma = lemmatizer.normal_forms(w_processed)[0]
            if lemma not in stopwords.words(DATASET_LANG):
                sentence.append(stemmer.stem(w_processed))
        if by_sentences:
            result.append(sentence)
        else:
            result += sentence
    return result

In [ ]:
def vectorize_tf_idf(text: str, matrix: np.matrix):
    preprocessed = preprocess_text(text)
    text_dict = count_words_in_docs([preprocessed])
    return tf_idf(text_dict, matrix)

In [ ]:
example_doc_1 = "В обеих странах есть партия войны . И в обеих странах эта война сейчас разжигается и поддерживается Смотрите, парни. И вот после этого руснявого пиздежа пидараны требуют к ним хорошего отношения? Мань, может это Украина хуярит по в на роисе градами ? Украина засылает в на роисю террористов бандитов ихтамнетов? Харк тебе в ебло, спидозная тварь."
example_doc_2 = "Возьмём как пример Россию, западноевропейские страны и США. Идёт метисация, сознательная политика замещения белого населения на пришлое черно-коричневое. Идёт создание новой расы метисов, исламизация и почернение. В крупных городах половина населения - выходцы из ебеней Мексики, Африки, Ближнего Востока, а в случае с Россией - Кавказа и Средней Азии. Этнические ниггеро-арабские гетто верят на хую законы как хотят, чудовищная по масштабам этническая преступность. Говорить о миграции и тем более затрагивать тему замещения коренного населения властями нельзя, иначе бутылка. Свобода слова тут не для вас, молодой человек. При этом говорить о том, что белые должны вымереть, и это нормально - можно. Белые официально вымирают ведётся пропаганда так или иначе направленная на снижение рождаемости белого населения. Феминизм, ЛГБТ, чайлдфри. Каждая женщина в Швеции - леволиберальная феминистка, это страна победившего феминизма. Что сегодня там происходит - страшно делается. Пропагандируются смешанные браки, межрасовые браки, пропагандируется превосходство детей-метисов. Идёт демонизация белых и пропаганда превосходства чёрных и смуглых мужчин, форс отношений белая женщина смуглый чёрный мужчина-мигрант. Как результат - всё больше чернильниц, всё больше смешанных браков, всё больше небелых метисов. Белые женщины просто не хотят контактировать с мужчинами своей нации и расы, наделяя их самыми плохими качествами и обожествляя черных. При этом большинство белых не считает завоз чурок чем-то плохим, наоборот, относятся к ним толерантно. Проводится политика насаждения толерантности, мультикультурализма, политкорректности и космополитизма. Набирающее популярность даже в России SJW - это вообще отдельная тема для обсуждения. Всё вышеперечисленное относится к сильнейшим когда-то странам, бывшим империям, нагибающим слабых. Сегодня происходит так, что бывшие империи в прямом смысле деградируют, вырождаются и вымирают, а место сильнейших когда-то, господствующих народов, занимают те, кого когда-то колонизировали. Во Франции к 2080 уже будут доминировать негры и арабы, в России - кавказцы и выходцы из средней Азии, в Великобритании - индийцы, негры, арабы, пакистанцы, etc. А в маленьких, нейтральных странах, вроде Словении или Беларуси, Литвы или Чехии, Румынии или Эстонии - всё пучком. Им вымирание не грозит, они остаются и будут оставаться белыми. Более того, у них ведётся политика, направленная на сохранение традиционных ценностей и культуры коренного населения. Они сказали беженцам нет . В Польшу, например, русскому или украинцу гораздо легче переехать и остаться, чем арабу или африканцу. В Германии ситуация противоположная, белых там не ждут. Польша, Чехия, Словакия, Венгрия, Словения, Хорватия, Сербия, БиГ, Черногория, Македония, Греция, Болгария, Румыния, Молдова, Украина, Беларусь, Литва, Латвия, Эстония - вот Европа будущего. Скандинавия, Южная, Западная Европа, а также Россия - лишатся коренного населения и своей культуры."

print(vectorize_tf_idf(example_doc_1, matrix))
print(vectorize_tf_idf(example_doc_2, matrix))

In [ ]:
def tokenize_word(word: str) -> str:
    stemmer = SnowballStemmer(DATASET_LANG)
    token = stemmer.stem(word)
    return token

In [ ]:
model_w2v = Word2Vec(sentences=[sentence for document in all_docs for sentence in document], epochs=50)
os.makedirs(MODEL_ARTIFACTS_DIR, exist_ok=True)
model_w2v.save(os.path.join(MODEL_ARTIFACTS_DIR, "w2v_weights__last"))

In [ ]:
model_w2v.wv.most_similar(tokenize_word("политика"))

In [ ]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [ ]:
cosine_similarity(
    model_w2v.wv[tokenize_word("Путин")],
    model_w2v.wv[tokenize_word("Европа")],
)

In [ ]:
def draw_words(terms, vectors_source):
    pca = PCA(n_components=2)
    vectors_2d = pd.DataFrame(pca.fit_transform([vectors_source[term] for term in terms]))
    vectors_2d.index = terms
    vectors_2d.columns = ["x", "y"]
    p = sns.scatterplot(data=vectors_2d, x="x", y="y")

    for i in vectors_2d.index:
        item = vectors_2d.loc[i]
        p.text(item.x, item.y, i)
    return p

In [ ]:
terms_to_check = [
    "политика",
    "США",
    "русский",
    "высказать",
    "оскорбления",
    "создавать",
    "Европу",
    "техника",
    "религии",
    "город",
    "Путин",
    "Трамп",
]

terms_to_check_tokenized = [tokenize_word(tok) for tok in terms_to_check]
draw_words(terms_to_check_tokenized, model_w2v.wv)

In [ ]:
def transform_to_compare(vectors: list[list[str]]) -> np.ndarray:
    pca = PCA(n_components=len(model_w2v.wv[0]))
    transformed = pca.fit_transform(vectors)
    return transformed

In [ ]:
def vectorize_tf_idf_tokens(matrix: list[str]) -> list[list[float]]:
    result = [None for _ in range(len(matrix.columns))]
    for i, token in tqdm(enumerate(matrix.columns)):
        result[i] = vectorize_tf_idf(token, matrix)
    
    return result

In [ ]:
terms_vectorized = [vectorize_tf_idf(token, matrix) for token in matrix.columns]

In [ ]:
to_cmp = transform_to_compare(terms_vectorized)

In [ ]:
terms_to_compare = pd.DataFrame(to_cmp)
terms_to_compare.index = matrix.columns

In [ ]:
def compare_methods(token_1, token_2):
    print(f"Косинусные расстояния между токенами {token_1} и {token_2} \nW2V:{cosine_similarity(model_w2v.wv[token_1], model_w2v.wv[token_2])}\nTf-Idf:{cosine_similarity(terms_to_compare.loc[token_1], terms_to_compare.loc[token_2])}\n\n\n")

In [ ]:
compare_methods(
    tokenize_word("Европа"),
    tokenize_word("США"),
)
compare_methods(
    tokenize_word("политика"),
    tokenize_word("политолог"),
)
compare_methods(
    tokenize_word("обезьяна"),
    tokenize_word("человек"),
)

In [ ]:
tfidf_data = {}
for i in range(len(matrix.columns)):
    tfidf_data[matrix.columns[i]] = terms_vectorized[i]

In [ ]:
draw_words(terms_to_check_tokenized, tfidf_data)

In [ ]:
def vectorize_w2v(sentences: list[list[str]], model_w2v):
    result_vec = np.zeros(model_w2v.vector_size)
    for s in sentences:
        sentence_vec = np.zeros(model_w2v.vector_size)
        for token in s:
            if model_w2v.wv.has_index_for(token):
                sentence_vec += model_w2v.wv[token]
        sentence_vec = sentence_vec / len(s) if len(s) > 0 else np.zeros(model_w2v.vector_size)
        result_vec += sentence_vec
    result_vec = result_vec / len(sentences) if len(sentences) > 0 else np.zeros(model_w2v.vector_size)
    return result_vec

In [ ]:
example_doc_1 = "В обеих странах есть партия войны . И в обеих странах эта война сейчас разжигается и поддерживается Смотрите, парни. И вот после этого руснявого пиздежа пидараны требуют к ним хорошего отношения? Мань, может это Украина хуярит по в на роисе градами ? Украина засылает в на роисю террористов бандитов ихтамнетов? Харк тебе в ебло, спидозная тварь."
example_doc_2 = "Возьмём как пример Россию, западноевропейские страны и США. Идёт метисация, сознательная политика замещения белого населения на пришлое черно-коричневое. Идёт создание новой расы метисов, исламизация и почернение. В крупных городах половина населения - выходцы из ебеней Мексики, Африки, Ближнего Востока, а в случае с Россией - Кавказа и Средней Азии. Этнические ниггеро-арабские гетто верят на хую законы как хотят, чудовищная по масштабам этническая преступность. Говорить о миграции и тем более затрагивать тему замещения коренного населения властями нельзя, иначе бутылка. Свобода слова тут не для вас, молодой человек. При этом говорить о том, что белые должны вымереть, и это нормально - можно. Белые официально вымирают ведётся пропаганда так или иначе направленная на снижение рождаемости белого населения. Феминизм, ЛГБТ, чайлдфри. Каждая женщина в Швеции - леволиберальная феминистка, это страна победившего феминизма. Что сегодня там происходит - страшно делается. Пропагандируются смешанные браки, межрасовые браки, пропагандируется превосходство детей-метисов. Идёт демонизация белых и пропаганда превосходства чёрных и смуглых мужчин, форс отношений белая женщина смуглый чёрный мужчина-мигрант. Как результат - всё больше чернильниц, всё больше смешанных браков, всё больше небелых метисов. Белые женщины просто не хотят контактировать с мужчинами своей нации и расы, наделяя их самыми плохими качествами и обожествляя черных. При этом большинство белых не считает завоз чурок чем-то плохим, наоборот, относятся к ним толерантно. Проводится политика насаждения толерантности, мультикультурализма, политкорректности и космополитизма. Набирающее популярность даже в России SJW - это вообще отдельная тема для обсуждения. Всё вышеперечисленное относится к сильнейшим когда-то странам, бывшим империям, нагибающим слабых. Сегодня происходит так, что бывшие империи в прямом смысле деградируют, вырождаются и вымирают, а место сильнейших когда-то, господствующих народов, занимают те, кого когда-то колонизировали. Во Франции к 2080 уже будут доминировать негры и арабы, в России - кавказцы и выходцы из средней Азии, в Великобритании - индийцы, негры, арабы, пакистанцы, etc. А в маленьких, нейтральных странах, вроде Словении или Беларуси, Литвы или Чехии, Румынии или Эстонии - всё пучком. Им вымирание не грозит, они остаются и будут оставаться белыми. Более того, у них ведётся политика, направленная на сохранение традиционных ценностей и культуры коренного населения. Они сказали беженцам нет . В Польшу, например, русскому или украинцу гораздо легче переехать и остаться, чем арабу или африканцу. В Германии ситуация противоположная, белых там не ждут. Польша, Чехия, Словакия, Венгрия, Словения, Хорватия, Сербия, БиГ, Черногория, Македония, Греция, Болгария, Румыния, Молдова, Украина, Беларусь, Литва, Латвия, Эстония - вот Европа будущего. Скандинавия, Южная, Западная Европа, а также Россия - лишатся коренного населения и своей культуры."

print("Пример 1: ", vectorize_w2v(preprocess_text(example_doc_1, True), model_w2v), end="\n\n\n")
print("Пример 2: ", vectorize_w2v(preprocess_text(example_doc_2, True), model_w2v))

In [ ]:
def save_embeddings(path: str, docs_paths: list[str]):
    vectorized_documents = {}

    for filename in tqdm(docs_paths):
        stems = get_stems_processed(filename)
        doc_id = "/".join(filename.split("/")[-2:]).replace(" ", "")

        vectorized_documents[doc_id] = vectorize_w2v(stems, model_w2v)


    with open(path, "w") as f:
        for k in vectorized_documents.keys():
            print(k.replace(".tsv", ""), *vectorized_documents[k], sep="\t", file=f)

In [ ]:
os.makedirs(DATASET_EMBEDDINGS_TRAIN_DIR, exist_ok=True)
os.makedirs(DATASET_EMBEDDINGS_TEST_DIR, exist_ok=True)

save_embeddings(os.path.join(DATASET_EMBEDDINGS_TRAIN_DIR, "train_embeddings.tsv"), docs_file_paths_train)
save_embeddings(os.path.join(DATASET_EMBEDDINGS_TEST_DIR, "test_embeddings.tsv"), docs_file_paths_test)
